<a href="https://colab.research.google.com/github/ayesha-ai-dev/my-python-program/blob/main/FYP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving review_polarity_dataset.csv to review_polarity_dataset.csv


In [ ]:
import pandas as pd
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv('review_polarity_dataset.csv')
df


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,review,polarity
0,"plot : two teen couples go to a church party ,...",neg
1,the happy bastard's quick movie review damn t...,neg
2,it is movies like these that make a jaded movi...,neg
3,""" quest for camelot "" is warner bros . ' firs...",neg
4,synopsis : a mentally unstable man undergoing ...,neg
...,...,...
1995,wow ! what a movie . it's everything a movie ...,pos
1996,"richard gere can be a commanding actor , but h...",pos
1997,"glory--starring matthew broderick , denzel was...",pos
1998,steven spielberg's second epic film on world w...,pos


# **Pre Processing**

In [ ]:
for i in range(len(df)):
    text = df.loc[i, 'review']

    text = text.lower()

    text = re.sub(r'[^a-z\s]', '', text)

    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    text = ' '.join(tokens)

    df.loc[i, 'cleaned'] = text

In [ ]:
df

In [ ]:
df.shape

(2000, 2)

# **Rule Based Predection**

In [ ]:

positive_words = ['love', 'great', 'good', 'best', 'perfect', 'enjoy', 'enjoyed', 'wonderful',
    'amazing', 'excellent', 'recommend']

negative_words = ['terrible', 'awful', 'worst', 'bad', 'hate', 'boring', 'waste',
    'poor', 'disappointing', 'mediocre', 'sad', 'lame', 'unhappy',
    'weak', 'cheap', 'slow', 'dirty', 'broken', 'pathetic', 'useless']


for i in range(len(df)):
    review_text = df.loc[i, 'cleaned']  # Get the cleaned review text


    pos_count = 0
    neg_count = 0


    for word in positive_words:
        if word in review_text:
            pos_count += 1


    for word in negative_words:
        if word in review_text:
            neg_count += 1


    if re.search(r"\bnot\s+(?:very\s+)?(?:good|great|excellent|amazing)\b", review_text):
        prediction = 'neg'
    else:

        if neg_count >= pos_count:
            prediction = 'neg'
        else:
            prediction = 'pos'


    df.loc[i, 'rule_based_pred'] = prediction


In [ ]:
df

,review,polarity,rule_based_pred
0,"plot : two teen couples go to a church party ,...",neg,neg
1,the happy bastard's quick movie review damn t...,neg,neg
2,it is movies like these that make a jaded movi...,neg,pos
3,""" quest for camelot "" is warner bros . ' firs...",neg,neg
4,synopsis : a mentally unstable man undergoing ...,neg,neg
...,...,...,...
1995,wow ! what a movie . it's everything a movie ...,pos,pos
1996,"richard gere can be a commanding actor , but h...",pos,neg
1997,"glory--starring matthew broderick , denzel was...",pos,pos
1998,steven spielberg's second epic film on world w...,pos,pos


# **LEXICON-BASED ANALYSIS (Valence Aware Dictionary and sEntiment Reasoner (VADER))**

In [ ]:

nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()
negation_pattern = r"\b(not|n't|didnt|don't|dont)\b\s+(?:very\s+)?(?:good|great|enjoy|love|like|recommend|best|well|better|perfect|unforgettable|worthwhile)\b"


lexicon_predictions = []

for i in range(len(df)):
      review_text = df.loc[i, 'cleaned']
      score = sia.polarity_scores(review_text)['compound']


      if re.search(negation_pattern, review_text):
        prediction = 'neg'
      elif score >= 0.05:
          prediction = 'pos'
      else:
          prediction = 'neg'


      lexicon_predictions.append(prediction)


df['lexicon_based_pred'] = lexicon_predictions


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
df

,review,polarity,lexicon_based_pred
0,"plot : two teen couples go to a church party ,...",neg,pos
1,the happy bastard's quick movie review damn t...,neg,pos
2,it is movies like these that make a jaded movi...,neg,pos
3,""" quest for camelot "" is warner bros . ' firs...",neg,neg
4,synopsis : a mentally unstable man undergoing ...,neg,pos
...,...,...,...
1995,wow ! what a movie . it's everything a movie ...,pos,pos
1996,"richard gere can be a commanding actor , but h...",pos,neg
1997,"glory--starring matthew broderick , denzel was...",pos,neg
1998,steven spielberg's second epic film on world w...,pos,pos


# **HYBRID APPROACH (VADER + PATTERNS)**

In [ ]:

nltk.download('vader_lexicon')


sia = SentimentIntensityAnalyzer()


df['hybrid_pred'] = ""


for i in range(len(df)):
    review_text = df.loc[i, 'cleaned']


    score = sia.polarity_scores(review_text)['compound']

    if score >= 0.05:
        vader_prediction = 'pos'
    else:
        vader_prediction = 'neg'


    if re.search(r"\b(not|never|no|lack|without)\s+\w+ful\b", review_text):
        hybrid_prediction = 'neg'


    elif re.search(r"\"?of course\"?|\"?sure\"?|\"?obviously\"?", review_text):
        if vader_prediction == 'pos':
            hybrid_prediction = 'neg'
        else:
            hybrid_prediction = vader_prediction

    elif re.search(r"\b(great|excellent|amazing|love|wonderful|highly recommend|best experience|perfect)\b", review_text):
        hybrid_prediction = 'pos'


    else:
        hybrid_prediction = vader_prediction


    df.loc[i, 'hybrid_pred'] = hybrid_prediction


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
df

,review,polarity,cleaned,hybrid_pred
0,"plot : two teen couples go to a church party ,...",neg,plot two teen couples go church party drink dr...,neg
1,the happy bastard's quick movie review damn t...,neg,happy bastards quick movie review damn yk bug ...,pos
2,it is movies like these that make a jaded movi...,neg,movies like make jaded movie viewer thankful i...,pos
3,""" quest for camelot "" is warner bros . ' firs...",neg,quest camelot warner bros first featurelength ...,neg
4,synopsis : a mentally unstable man undergoing ...,neg,synopsis mentally unstable man undergoing psyc...,neg
...,...,...,...,...
1995,wow ! what a movie . it's everything a movie ...,pos,wow movie everything movie funny dramatic inte...,pos
1996,"richard gere can be a commanding actor , but h...",pos,richard gere commanding actor hes always great...,pos
1997,"glory--starring matthew broderick , denzel was...",pos,glorystarring matthew broderick denzel washing...,neg
1998,steven spielberg's second epic film on world w...,pos,steven spielbergs second epic film world war i...,pos


In [ ]:

actual = df['polarity']


predicted = df['rule_based_pred']


print(classification_report(actual, predicted))
print("Confusion Matrix:")
print(confusion_matrix(actual, predicted))


predicted = df['lexicon_based_pred']

print(classification_report(actual, predicted))
print("Confusion Matrix:")
print(confusion_matrix(actual, predicted))


predicted = df['hybrid_pred']

print(classification_report(actual, predicted))
print("Confusion Matrix:")
print(confusion_matrix(actual, predicted))



=== Rule-Based ===
              precision    recall  f1-score   support

         neg       0.74      0.67      0.70      1000
         pos       0.70      0.76      0.73      1000

    accuracy                           0.71      2000
   macro avg       0.72      0.71      0.71      2000
weighted avg       0.72      0.71      0.71      2000

Confusion Matrix:
[[669 331]
 [240 760]]

=== Lexicon-Based ===
              precision    recall  f1-score   support

         neg       0.68      0.44      0.53      1000
         pos       0.59      0.80      0.67      1000

    accuracy                           0.62      2000
   macro avg       0.63      0.62      0.60      2000
weighted avg       0.63      0.62      0.60      2000

Confusion Matrix:
[[439 561]
 [205 795]]

=== Hybrid Rule+Pattern ===
              precision    recall  f1-score   support

         neg       0.52      0.52      0.52      1000
         pos       0.52      0.52      0.52      1000

    accuracy                